# Example: Parallel Network using LFPy

This example notebook submits a job to a chosen compute cluster (**JUSUF**), loads a **Singularity** container with a working **LFPy** installation and executes a version of the LFPy example file ``example_network.py`` (https://github.com/LFPy/LFPy/blob/master/examples/example_network/example_network.py) in parallel on one HPC backend of EBRAINS/HBP.

The example demonstrates usage of ``LFPy.Network`` with network of ball-and-stick type
morphologies with active HH channels inserted in the somas and passive-leak
channels distributed throughout the apical dendrite. The corresponding
morphology and template specifications are in the files ``BallAndStick.hoc`` and
``BallAndStickTemplate.hoc``.


For more indepth info on using HPC backends via EBRAINS see https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

In [206]:
!pwd

/mnt/user/drive/Shared with groups/LFPy showcase


## Prepare job

In [5]:
# use the pyunicore library
!pip install pyunicore --upgrade

Requirement already up-to-date: pyunicore in /opt/app-root/lib/python3.6/site-packages (0.8.0)
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [6]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint
from IPython.display import IFrame

In [7]:
# Create connection to supercomputer (i.e., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [8]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

{'RWTH': 'https://unicore.hpc.itc.rwth-aachen.de:8080/RWTH/rest/core',
 'FZJ_JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/FZJ_JURECA/rest/core',
 'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core',
 'irene': 'https://hbp-unicore.ccc.cea.fr/irene/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core',
 'CINECA-GALILEO': 'https://grid.hpc.cineca.it:9111/CINECA-GALILEO/rest/core'}

In [9]:
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    # cluster may be dropped from Registry.site_urls for whatever reason
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [10]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers = headers, verify = False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

Status code 200 
Content-type application/json 


## Download LFPy example network files
Prepare simulation files using example files from the main LFPy repository (https://github.com/LFPy/LFPy.git)

In [155]:
!wget -O example_network.py https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/example_network.py
!wget -O BallAndStick.hoc https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStick.hoc
!wget -O BallAndStickTemplate.hoc https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStickTemplate.hoc

--2020-10-26 12:50:31--  https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/example_network.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18899 (18K) [text/plain]
Saving to: ‘example_network.py’

100%[======================================>] 18,899      --.-K/s   in 0.01s   

2020-10-26 12:50:31 (1.62 MB/s) - ‘example_network.py’ saved [18899/18899]

--2020-10-26 12:50:32--  https://raw.githubusercontent.com/LFPy/LFPy/master/examples/example_network/BallAndStick.hoc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 610 [text/plain]
Saving to: ‘BallAndStick.hoc’

100%[============

In [156]:
## to make things more interesting with HPCs, let's increase the network size and adjust synaptic weights by applying a patch:
diff = '''240c240
< population_sizes = [80, 20]
---
> population_sizes = [1024, 256]
257,260c257,260
< weightArguments = [[dict(loc=0.002, scale=0.0002),
<                     dict(loc=0.002, scale=0.0002)],
<                    [dict(loc=0.02, scale=0.002),
<                     dict(loc=0.02, scale=0.002)]]
---
> weightArguments = [[dict(loc=0.0002, scale=0.00002),
>                     dict(loc=0.0002, scale=0.00002)],
>                    [dict(loc=0.002, scale=0.0002),
>                     dict(loc=0.002, scale=0.0002)]]
380c380
<             ax.plot(t[t >= 200], g[t >= 200], '|', label=name)
---
>             ax.plot(t[t >= 200], g[t >= 200], '.', ms=2, label=name)
'''

with open('patch.diff', 'w') as f:
    f.writelines(diff)

In [157]:
# apply patch
!patch example_network.py patch.diff

patching file example_network.py


## Prepare singularity container
See https://gitlab.version.fz-juelich.de/bvonstvieth_publications/container_userdoc_tmp for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/LFPy/LFPydebian/main/mpich.Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job
This step combines in a single session the following:

- download LFPy container
- upload simulation files from the collab
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation
- zip simulation output

In [188]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = ["example_network.py", "BallAndStick.hoc", "BallAndStickTemplate.hoc"] 

In [189]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    "CPUs": "256",
    "Runtime": "600",
}

In [191]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
sib download --recipe-name lfpydebian_mpich
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [192]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node 
srun singularity exec lfpydebian_mpich.sif python3 -u example_network.py
"""

In [193]:
# commands run after job is done
simulation_job["User postcommand"] = "tar -cvf example_network_output.tar example_network_output" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [194]:
simulation_job

{'Resources': {'Queue': 'batch', 'CPUs': '256', 'Runtime': '600'},
 'User precommand': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nsib download --recipe-name lfpydebian_mpich\n',
 'RunUserPrecommandOnLoginNode': 'true',
 'Executable': 'unset DISPLAY  # matplotlib may look for a nonexistant display on compute node \nsrun singularity exec lfpydebian_mpich.sif python3 -u example_network.py\n',
 'User postcommand': 'tar -cvf example_network_output.tar example_network_output',
 'RunUserPostcommandOnLoginNode': 'true'}

In [195]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [196]:
# wait while job is running
while job.is_running():
    sleep(10)

In [197]:
job.working_dir.listdir().keys()

dict_keys(['example_network_output/', 'stderr', 'lfpydebian_mpich.sif', 'BallAndStickTemplate.hoc', 'example_network.py', 'example_network_output.tar', 'stdout', '.UNICORE_POST_0/', '.UNICORE_PRE_0/', 'UNICORE_Job_1603714221554', 'UNICORE_SCRIPT_EXIT_CODE', 'BallAndStick.hoc', 'bss_submit_1603714221554'])

In [201]:
# STDERR output (if any)
stderr = job.working_dir.stat('stderr')
pprint(stderr.raw().readlines()[:5])
pprint(stderr.raw().readlines()[-5:])

[b'/usr/local/lib/python3.7/dist-packages/LFPy/cell.py:185: UserWarning: 0 sect'
 b"ions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n',
 b'/usr/local/lib/python3.7/dist-packages/LFPy/cell.py:185: UserWarning: 0 sect'
 b"ions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n',
 b'/usr/local/lib/python3.7/dist-packages/LFPy/cell.py:185: UserWarning: 0 sect'
 b"ions detected! Consider setting 'delete_sections=True'\n"]
[b'  warn(mssg)\n',
 b'/usr/local/lib/python3.7/dist-packages/LFPy/cell.py:185: UserWarning: 8 sect'
 b"ions detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n',
 b'/usr/lib/python3/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning'
 b': Using a non-tuple sequence for multidimensional indexing is deprecated; us'
 b'e `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interp'
 b'reted as an array index, `arr[np.array(seq)]`, which will result either in a'
 b'n error or a differen

In [199]:
# STDOUT output
stdout = job.working_dir.stat('stdout')
pprint(stdout.raw().readlines()[:10])
pprint(stdout.raw().readlines()[-10:])

[b'numprocs=256\n',
 b'Connected population E to E by 104836 connections and 160662 synapses\n',
 b'Connected population E to I by 26527 connections and 40752 synapses\n',
 b'Connected population I to E by 26287 connections and 117854 synapses\n',
 b'Connected population I to I by 6562 connections and 29530 synapses\n',
 b'Adjusting r-distance to soma segments\n',
 b'Adjusting r-distance to soma segments\n',
 b'Adjusting r-distance to soma segments\n',
 b'Adjusting r-distance to soma segments\n',
 b'Adjusting r-distance to soma segments\n']
[b't = 300.0 ms\n',
 b't = 400.0 ms\n',
 b't = 500.0 ms\n',
 b't = 600.0 ms\n',
 b't = 700.0 ms\n',
 b't = 800.0 ms\n',
 b't = 900.0 ms\n',
 b't = 1000.0 ms\n',
 b't = 1100.0 ms\n',
 b't = 1200.0 ms\n']


In [200]:
# download simulation output
job.working_dir.stat('example_network_output.tar').download('example_network_output.tar')

In [202]:
# kill job, clean up files on the remote
job.delete()

## Investigate simulation output

In [203]:
# untar output to folder example_network_output
!tar -xf example_network_output.tar

In [204]:
# quick look at e.g., the extracellular potential and spike raster plot
IFrame("./example_network_output/extracellular_potential.pdf", width=800, height=600)

In [205]:
IFrame("./example_network_output/spike_raster.pdf", width=800, height=600)